In [1]:
# Written Observations:

#1) Overall students have a higher reading scores than math scores. This could be due to way questions are asked on tests.

#2) Schools that have a higher per student budget actually performed worse than schools with the lowest per student budget.

#3) From the data we can see that charter high school are better acdemically and have a higher overall passing percentage. 
#The top 5 performing schools were also charter schools.


In [2]:
!pip install pandas

In [3]:
import pandas as pd

In [4]:
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"
#reading into files
school_df = pd.read_csv(school_csv, encoding="utf-8")
student_df = pd.read_csv(student_csv, encoding="utf-8")
#merging files
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
#displays df
merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#Getting count of "total schools" using .shape[] function to counts rows
schools_df = school_df.shape[0]
#counting all of the students from the merged_df
students_t = merged_df["student_name"].count()
#summing the "budget" column from the school_df
budget_t = school_df["budget"].sum()
#finding average "math_score" from the merged_df
m_score_avg = merged_df["math_score"].mean()
#finding average "reading_score" from the merged_df
r_score_avg = merged_df["reading_score"].mean()
#finding all math scores above or equal to 70 in the merged_df
m_pass = merged_df.loc [merged_df["math_score"] >=70]
#finding all reading scores above or equal to 70 in the merged_df
r_pass = merged_df.loc [merged_df["reading_score"] >=70]
#Finding '% passing math' by taking the length of 'm_pass' and dividing by the length of  merged-df using .shape[] function and multiply by 100
perc_m_pass = (m_pass.shape[0]/merged_df.shape[0])*100
#Finding '% passing reading' by taking the length of 'r_pass' and dividing by the length of  merged-df using .shape[] function and multiply by 100
perc_r_pass = (r_pass.shape[0]/merged_df.shape[0])*100
#Finding 'overall passing' by finding all instances where both reading AND math scores for one student were above or equal to 70 in the merged_df
o_pass = merged_df.loc [(merged_df["reading_score"] >=70)&(merged_df["math_score"] >=70)]
#Finding '% overall passing ' by taking the length of 'o_pass' and dividing by the length of  merged-df using .shape[] function and multiply by 100
perc_o_pass = (o_pass.shape[0]/merged_df.shape[0])*100

#Putting all lists into a DataFrame
district_summary_df = pd.DataFrame({"Total Schools": [schools_df],
                                    "Total Students": [students_t],
                                    "Total Budget": [budget_t],
                                    "Average Math Score":[m_score_avg],
                                    "Average Reading Score": [r_score_avg],
                                    "% Passing Math": [perc_m_pass],
                                    "% Passing Reading":[perc_r_pass],
                                   "% Overall Passing": [perc_o_pass]})
#Formatting '$' for 'Total Budget' output
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
#displays df
district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [6]:
#Modifying school_df by dropping "School ID" column and setting 'school_name' as index, created new df to better work with
new_school_df = school_df.drop(columns="School ID").set_index('school_name')
#Created new df from student_df grouped by 'school_name'
new_student_df = student_df.groupby('school_name')
#On merged document, grouping by "school_name" and counting all students
total = merged_df.groupby("school_name").count()
#creating list of school types from new_school_df
s_type = new_school_df["type"]
#creating list of school size from new_school_df
size = new_school_df["size"]
#creating list of school budgets from new_school_df and formatting '$'
budget = new_school_df["budget"].map('${:,.2f}'.format)
#finding per student budget from new_school_df by dividng the school budget by school size and formatting '$'
per_student = (new_school_df["budget"]/new_school_df["size"]).map('${:,.2f}'.format)
#finding 'math_score' average from new student df
math_avg = new_student_df["math_score"].mean()
#finding 'reading_score' average from new student df
read_avg = new_student_df["reading_score"].mean()
#finding '% passing math' by using 'm_pass' to groupby 'school_name' and count 'student name' then divide by total rows in 'math_score' and multiply by 100
math_percent = (m_pass.groupby(["school_name"]).count()["student_name"]/total['math_score'])*100
#finding '% passing reading' by using 'r_pass' to groupby 'school_name' and count 'student name' then divide by total rows in 'reading_score' and multiply by 100
read_percent = (r_pass.groupby(["school_name"]).count()["student_name"]/total['reading_score'])*100

#Finding 'overall passing' by finding all instances where both reading AND math scores for one student were above or equal to 70 in the student_df, grouping by 'school_name' then counting 'student_name'
over_pass = student_df[(student_df['math_score']>=70) & (student_df['reading_score']>=70)].groupby(["school_name"]).count()["student_name"]
#Finding '% overall passing' by taking 'over_pass' and dividing by total rows in 'math_score' and multiply by 100
over_percent = (over_pass/total['math_score'])*100

#Putting all lists into a df
school_summary_df = pd.DataFrame({"School Type": s_type,
                                     "Total Students": size,
                                     "Total School Budget": budget,
                                     "Per Student Budget": per_student,
                                     "Average Math Score":math_avg,
                                     "Average Reading Score": read_avg,
                                     "% Passing Math": math_percent,
                                     "% Passing Reading": read_percent,
                                     "% Overall Passing": over_percent})
#displays df
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [7]:
#Going into school_summary_df, sorting by % Overall Passing, displaying only first five in list
top_5 = school_summary_df.sort_values(by=["% Overall Passing"],ascending=False).head()
#displays df
top_5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [8]:
#Going into school_summary_df, sorting by % Overall Passing, ascending set to default, displaying only lowest five in list
bot_5 = school_summary_df.sort_values(by=["% Overall Passing"]).head()
#displays df
bot_5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [9]:
#Pulling unique inputs in "grades" columns from student_df
student_df["grade"].unique()

#Locating all 9th graders in student_df
grade_9 = student_df.loc[student_df["grade"] == "9th",:]
#Grouping all 9th graders by school name 
total_9_per_school = grade_9.groupby(["school_name"])
#Taking the average "math score" per school
avg_9_math = total_9_per_school["math_score"].mean()

#Repeat comments for grade 10-12


grade_10 = student_df.loc[student_df["grade"] == "10th",:]
total_10_per_school = grade_10.groupby(["school_name"])
avg_10_math = total_10_per_school["math_score"].mean()

grade_11 = student_df.loc[student_df["grade"] == "11th",:]
total_11_per_school = grade_11.groupby(["school_name"])
avg_11_math = total_11_per_school["math_score"].mean()

grade_12 = student_df.loc[student_df["grade"] == "12th",:]
total_12_per_school = grade_12.groupby(["school_name"])
avg_12_math = total_12_per_school["math_score"].mean()

#Putting all lists into a df
math_scores_df = pd.DataFrame({"9th Grade": avg_9_math,
                               "10th Grade": avg_10_math,
                               "11th Grade": avg_11_math,
                               "12th Grade": avg_12_math})
#displays df
math_scores_df

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [10]:
#Locating all 9th graders in student_df
grade_9 = student_df.loc[student_df["grade"] == "9th",:]
#Grouping all 9th graders by school name 
total_9_per_school = grade_9.groupby(["school_name"])
#Taking the average "reading score" per school
avg_9_math = total_9_per_school["reading_score"].mean()


#Repeat comments for grade 10-12
grade_10 = student_df.loc[student_df["grade"] == "10th",:]
total_10_per_school = grade_10.groupby(["school_name"])
avg_10_math = total_10_per_school["reading_score"].mean()

grade_11 = student_df.loc[student_df["grade"] == "11th",:]
total_11_per_school = grade_11.groupby(["school_name"])
avg_11_math = total_11_per_school["reading_score"].mean()

grade_12 = student_df.loc[student_df["grade"] == "12th",:]
total_12_per_school = grade_12.groupby(["school_name"])
avg_12_math = total_12_per_school["reading_score"].mean()

#Putting all lists into a df
reading_scores_df = pd.DataFrame({"9th Grade": avg_9_math,
                               "10th Grade": avg_10_math,
                               "11th Grade": avg_11_math,
                               "12th Grade": avg_12_math})
#displays df
reading_scores_df

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [24]:
#creating copy of school_summary_df 
one_df = school_summary_df.copy() 
one_df.astype({'Per Student Budget': 'float64'}).dtypes
#setting bins and bin_labels, cutting data from the school_summary_df "Per Student Budget" column
bins = (0, 584, 629, 644, 675)
bin_labels = ["<$584", "$585-629", "$630-644", "$645-675"]

one_df.dtypes
#range_bins_df = pd.cut(one_df["Per Student Budget"], bins, labels=bin_labels)


#adding new column 'range_bins_df' to school_summary_df as 'Spending Range (Per Student)'


#one_df["Spending Ranges (Per Student)"] = range_bins_df
#modifying df and setting columns to disply their averages


#one_df = one_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()

#displays df
#one_df

ValueError: could not convert string to float: '$628.00'

In [ ]:
#creating copy of school_summary_df 
two_df = school_summary_df.copy()
#setting bins and bin_labels, cutting data from the school_summary_df "Total Students" column
bins = (0, 1000, 2000, 5000)
bin_labels = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]
size_bins_df = pd.cut(two_df["Total Students"], bins, labels=bin_labels)


#adding new column 'size_bins_df' to school_summary_df as 'Spending Range (Per Student)'
two_df["School Size"] = size_bins_df
#modifying df and setting columns to disply their averages
two_df = two_df.groupby(["School Size"])["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()
#displays df
two_df


In [ ]:
#creating copy of school_summary_df 
three_df = school_summary_df.copy()
#modifying df and setting columns to disply their averages
three_df = school_summary_df.groupby(["School Type"])["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()

#displays df
three_df